# Climate Action Reserve
https://www.climateactionreserve.org

In [ ]:
import json
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import transform
from tqdm import tqdm
import time
import os
from random import uniform

### Load data

In [ ]:
df = pd.read_csv("../input/CAR/car_export_02_28_2025.csv", encoding="ISO-8859-1")
df.head(1)

In [ ]:
df['project_id'] = df['Project ID'].str.extract(r'CAR(\d+)')

In [ ]:
df = (df.loc[(df['Project Type'] == 'Reforestation - ARB Compliance') | 
              (df['Project Type'] == 'Reforestation')].copy())

In [ ]:
df.project_id.unique()

Check and, if applicable, retrieve shapefiles from project websites

In [ ]:
gdf = gpd.GeoDataFrame()

for project_id in tqdm(df['project_id'].unique().tolist()):

    shp_dir = f"/Users/tillkoebe/Documents/GitHub/Forest_Monitoring/input/CAR/1957/project_area.shp"
    
    if os.path.exists(shp_dir):
        temp = gpd.read_file(shp_dir)
    
        invalid_geom = False
        if len(temp[temp.geometry.is_valid == False]) > 0:
            invalid_geom = True  
    
        temp['geometry'] = temp['geometry'].apply(lambda geometry: transform(lambda x, y, z=None: (x, y), geometry))

        while 'MultiPolygon' in temp.geometry.geom_type.unique().tolist():
            temp = temp.explode(index_parts=False)
            temp['geometry'] = temp['geometry'].make_valid()

        temp = temp.loc[temp.geometry.geom_type.isin(['Polygon', 'Point'])].copy()
        
        temp = temp.to_crs(4326)

        temp['project_id'] = project_id
        temp['site_sqkm'] = temp.to_crs(3857).area/1e6
        temp['project_geometries_invalid'] = invalid_geom
    
        temp = df.merge(temp, on = 'project_id', how = 'inner')
        temp = temp[['Project ID', 'Project Listed Date', 'geometry', 'site_sqkm', 'project_geometries_invalid']]
    
    else:
        temp = (df
                .loc[df.project_id == project_id, ['Project ID', 'Project Listed Date']]
                .assign(geometry=None, site_sqkm=None, project_geometries_invalid=None))
    
    temp['url'] = 'https://thereserve2.apx.com/mymodule/reg/TabDocuments.asp?r=111&ad=Prpt&act=update&type=PRO&aProj=pub&tablename=doc&id1='+project_id

    temp['project_pdf_available'] = False

    pd_dir = f"/Users/tillkoebe/Documents/GitHub/Forest_Monitoring/midsave/project_descriptions/pd_car_{project_id}.pdf"
    if os.path.exists(pd_dir):
        temp['project_pdf_available'] = True

    
    gdf = pd.concat([gdf, temp], ignore_index= True)

In [ ]:
gdf = gpd.GeoDataFrame(gdf, geometry = 'geometry', crs = 'EPSG:4326')

In [ ]:
gdf.reset_index(names = ['site_id_created'], inplace = True)

In [ ]:
gdf.rename(columns = {'Project ID':'project_id_reported','Project Listed Date':'planting_date_reported'}, inplace = True)

In [ ]:
gdf = gdf.assign(species_count_reported=None,
              species_planted_reported=None,
              survival_rate_reported=None,
              trees_planted_reported=None)

In [ ]:
gdf

### Save it

In [ ]:
gdf.to_file('../midsave/climate_action_reserve.gpkg', driver='GPKG')